In [1]:
import requests
result = requests.get("https://rickandmorty.fandom.com/sitemap-newsitemapxml-index.xml")
c = result.content

In [2]:
from bs4 import BeautifulSoup 
#import lxml
import xml.etree.ElementTree as ET
root = ET.fromstring(c)
for link in root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
    print(link.text)

https://rickandmorty.fandom.com/sitemap-newsitemapxml-NS_0-id-1461-30925.xml
https://rickandmorty.fandom.com/sitemap-newsitemapxml-NS_14-id-1509-30609.xml
https://services.fandom.com/discussions-sitemap/sitemap/881799


In [3]:
elements = dict()
for page in root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
    result = requests.get(page.text)
    c = result.content
    new_root = ET.fromstring(c)
    for element in new_root.iter('{http://www.sitemaps.org/schemas/sitemap/0.9}loc'):
        elements[element.text.split('/')[-1]] = element.text
print('Found {} elements'.format(len(elements)))

Found 4119 elements


In [4]:
elements

{'Rickipedia': 'https://rickandmorty.fandom.com/wiki/Rickipedia',
 'Rick_Sanchez': 'https://rickandmorty.fandom.com/wiki/Rick_Sanchez',
 'Morty_Smith': 'https://rickandmorty.fandom.com/wiki/Morty_Smith',
 'Summer_Smith': 'https://rickandmorty.fandom.com/wiki/Summer_Smith',
 'Rick_and_Morty_(TV_series)': 'https://rickandmorty.fandom.com/wiki/Rick_and_Morty_(TV_series)',
 'Anatomy_Park_(episode)': 'https://rickandmorty.fandom.com/wiki/Anatomy_Park_(episode)',
 'Season_1': 'https://rickandmorty.fandom.com/wiki/Season_1',
 'Anatomy_Park_(location)': 'https://rickandmorty.fandom.com/wiki/Anatomy_Park_(location)',
 'Lawnmower_Dog': 'https://rickandmorty.fandom.com/wiki/Lawnmower_Dog',
 'Pilot': 'https://rickandmorty.fandom.com/wiki/Pilot',
 'M._Night_Shaym-Aliens!': 'https://rickandmorty.fandom.com/wiki/M._Night_Shaym-Aliens!',
 'Meeseeks_and_Destroy': 'https://rickandmorty.fandom.com/wiki/Meeseeks_and_Destroy',
 'Rick_Potion_No._9': 'https://rickandmorty.fandom.com/wiki/Rick_Potion_No._9',


In [5]:
import codecs
f = codecs.open("R_M_relations.csv", "a","utf-8-sig")
f.write('src,dest\n')
for k,v in elements.items():
    #prende la pagina html e la parsa
    result = requests.get(v)
    c = result.content
    soup = BeautifulSoup(c, "html.parser")  # parsa la pagina HTML
    #nell'intero documento html prende solo i <div> che hanno la classe specificata come secondo parametro
    soup.find_all('div' ,class_='pi-item pi-data pi-item-spacing pi-border-color')
    famRel=[]
    #tra i <div> risultanti va a prendere solo quelli relativi alla famiglia o alle relazioni
    for elem in soup.find_all('div' ,class_='pi-item pi-data pi-item-spacing pi-border-color'):
        first_child=elem.contents
        if 'FAMILY' in str(first_child):
            famRel.append(first_child)
        elif 'RELATIONSHIPS' in str(first_child):
            famRel.append(first_child)
    #accoda i div risultanti
    new_page=''
    for el in famRel:
        for e in el:
            if 'div' in str(e):
                new_page=new_page+str(e)
    #si trasforma il risultato in una "pagina html" per navigare più facilmente
    persons = BeautifulSoup(new_page, "html")
    persons_list=persons.find_all('a')
    for per in persons_list:
        #una volta prese le persone della famiglia e delle relazioni le trascrive sul csv
        if not '[' in per.contents[0]:
            f.write(k.replace("_", " ").replace("%27","'")+','+per.contents[0].replace(",","").removesuffix(' (C-137)')+'\n')
f.close()

In [4]:
import codecs
f2 = codecs.open("R_M_characters.csv", "a","utf-8-sig")
f2.write('name,species\n')
for k,v in elements.items():
    #prende la pagina html e la parsa
    result2 = requests.get(v)
    c2 = result2.content
    soup2 = BeautifulSoup(c2, "html")  # parsa la pagina HTML
    for elem in soup2.find_all('div' ,class_='pi-item pi-data pi-item-spacing pi-border-color'):
        first_child=elem.contents
        if 'SPECIES' in str(first_child):
            f2.write(k.replace("_", " ").replace("%27","'")+',')
            for el in first_child:
                appo=''
                if 'div' in str(el):
                    for e in el:
                        appo=appo+str(e.text)
                    f2.write(appo.replace(',','')+'\n')
f2.close()